In [1]:
#testing pyspark installation
import findspark
findspark.init()
findspark.find()
import pyspark
findspark.find()

'C:\\spark-3.0.3-bin-hadoop2.7'

In [2]:
#initiate spark context

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = pyspark.SparkConf().setAppName('SparkApp').setMaster('local')
sc = pyspark.SparkContext(conf=conf)
spark = SparkSession.builder.getOrCreate()

In [3]:
spark

In [4]:
df1 = spark.read.csv('../Dataset_rev/watchedVideo_rev.csv', header=True, inferSchema=True)
df2 = spark.read.csv('../Dataset/feeds.csv', header=True, inferSchema=True)

In [5]:
df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- os: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- videoId: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- appVersion: string (nullable = true)
 |-- dateTime: string (nullable = true)
 |-- feedId: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- user_city: string (nullable = true)
 |-- user_state: string (nullable = true)
 |-- user_country: string (nullable = true)



In [6]:
# renaming
df1 = df1.drop('_c0').withColumnRenamed("user_city", "watched_city")\
.withColumnRenamed("user_state", "watched_state").withColumnRenamed("user_country", "watched_country")

In [7]:
df2.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- id: string (nullable = true)
 |-- name: string (nullable = true)



In [8]:
#left join feeds and wathedVideo
df3 = df1.join(df2.drop("_c0"), df1.feedId == df2.id, "left")
df3 = df3.withColumnRenamed("name", "feed_name")

In [9]:
df3.printSchema()

root
 |-- os: string (nullable = true)
 |-- duration: double (nullable = true)
 |-- videoId: string (nullable = true)
 |-- userId: string (nullable = true)
 |-- appVersion: string (nullable = true)
 |-- dateTime: string (nullable = true)
 |-- feedId: string (nullable = true)
 |-- latitude: double (nullable = true)
 |-- longitude: double (nullable = true)
 |-- watched_city: string (nullable = true)
 |-- watched_state: string (nullable = true)
 |-- watched_country: string (nullable = true)
 |-- id: string (nullable = true)
 |-- feed_name: string (nullable = true)



### feedId -> feed_name

In [10]:
df3.drop("feedId", "id").write.options(header='True', delimiter=',').csv("watchedVideo_rev2")

In [11]:
#stop
sc.stop()